# day 36


In [141]:
import tensorflow as tf
import numpy as np

In [142]:
sample = " Mulmi team, cheer-up"
# " Mulmi team, cheer-u" => " Mulmi team, cheer-up"


In [143]:
set(sample) # 중복이 제거된상태로 도출
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}
char2idx

{',': 0,
 'c': 1,
 'i': 2,
 't': 3,
 '-': 4,
 'e': 5,
 ' ': 6,
 'M': 7,
 'r': 8,
 'p': 9,
 'h': 10,
 'm': 11,
 'u': 12,
 'a': 13,
 'l': 14}

In [144]:
dic_size = len(char2idx)
hidden_size = num_classes = dic_size
# dic_size : ohe 크기 ( rnn input size )
# hidden_size = output size
# num_classes : 최종 output size ( softmax )
batch_size =1 # 데이터 1개 - 배치 - 업데이트  
sequence_length = len(sample) - 1
learning_rate = 0.1


In [145]:
num_classes

15

In [146]:
tf.reset_default_graph()
sample_idx=[char2idx[c] for c in sample]
#sample=" Mulmi team, cheer-up"
xdata=[sample_idx[:-1]]
ydata=[sample_idx[1:]]
x=tf.placeholder(tf.int32, [None,sequence_length])
y=tf.placeholder(tf.int32, [None,sequence_length])
x_one_hot=tf.one_hot(x,num_classes) # 클래스의 개수
cell=tf.contrib.rnn.BasicLSTMCell(
num_units=hidden_size)
ini_state=cell.zero_state(batch_size, tf.float32)
outputs, _states=tf.nn.dynamic_rnn(cell, x_one_hot, 
                                   initial_state=ini_state, dtype=tf.float32)

In [147]:
xfc=tf.reshape(outputs, [-1, hidden_size])
outputs=tf.contrib.layers.fully_connected(xfc, num_classes, activation_fn=None)
outputs=tf.reshape(outputs,[batch_size,sequence_length, num_classes])
weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction=tf.argmax(outputs, axis=2)

In [148]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        cv, _=sess.run([loss, train], feed_dict={x:xdata, y:ydata})
        res=sess.run(prediction, feed_dict={x:xdata})
        res_str=[idx2char[c] for c in np.squeeze(res)]
        print(i,"loss:", cv, "prediction:", "".join(res_str))
        

0 loss: 2.696514 prediction: euuu  eeuuuueeuuuuuu
1 loss: 2.5071232 prediction: eueeeeeeeeeeeeeeuuuu
2 loss: 2.232521 prediction: eumm  eemmm eeeeeeee
3 loss: 2.1539896 prediction: eu m   emm   eeeruup
4 loss: 1.697712 prediction: eu m   emm   eeeruup
5 loss: 1.4379492 prediction: eumm  temm   heer-up
6 loss: 1.136668 prediction: eumm  team,  heer-up
7 loss: 0.8497551 prediction: eulm  team, cheer-up
8 loss: 0.61186945 prediction: eulmi team, cheer-up
9 loss: 0.44376048 prediction: eulmi team, cheer-up
10 loss: 0.31256527 prediction: Mulmi team, cheer-up
11 loss: 0.22132596 prediction: Mulmi team, cheer-up
12 loss: 0.15985098 prediction: Mulmi team, cheer-up
13 loss: 0.11295144 prediction: Mulmi team, cheer-up
14 loss: 0.077970006 prediction: Mulmi team, cheer-up
15 loss: 0.0546977 prediction: Mulmi team, cheer-up
16 loss: 0.038328953 prediction: Mulmi team, cheer-up
17 loss: 0.027649816 prediction: Mulmi team, cheer-up
18 loss: 0.020540832 prediction: Mulmi team, cheer-up
19 loss: 0.0

## apriori  구현

In [149]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


In [150]:
dataset = [['사과','치즈','생수'],
           ['생수','호두','치즈','고등어'],
           ['수박','사과','생수'],
           ['생수','호두','치즈','옥수수']]


In [151]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary



array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [152]:
df = pd.DataFrame(te_ary, columns = te.columns_)
df
freq_item = apriori(df, min_support = 0.5, use_colnames=True)
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(생수, 호두)"
7,0.50,"(치즈, 호두)"
8,0.50,"(생수, 치즈, 호두)"


In [153]:
from mlxtend.frequent_patterns import association_rules
association_rules(freq_item, metric='lift', min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
1,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
2,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
3,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
4,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
5,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
6,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
7,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
8,"(생수, 치즈)",(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
9,"(생수, 호두)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf


## 연습문제
### 제공받은 파일에서 연관규칙 도출
### lift 기준으로 상위 10개 규칙을 출력

#### grocery item

In [154]:
# parsing error
# data = pd.read_csv('data/groceries.csv',error_bad_lines=False)
# data
import re

groc = list()
with open('data/groceries.csv','r') as fp:
    while True:
        line = fp.readline()
        if not line : break
        line = re.sub('\n','', line)
        groc.append(line.split(','))
            

In [155]:
te = TransactionEncoder()
groc_mat = te.fit_transform(groc)
print(groc_mat)


[[False False False ... False False False]
 [False False False ... False  True False]
 [False False False ...  True False False]
 ...
 [False False False ... False  True False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [156]:
groc_df = pd.DataFrame(groc_mat, columns = te.columns_)
print(groc_df)

      Instant food products  UHT-milk  abrasive cleaner  artif. sweetener  \
0                     False     False             False             False   
1                     False     False             False             False   
2                     False     False             False             False   
3                     False     False             False             False   
4                     False     False             False             False   
5                     False     False              True             False   
6                     False     False             False             False   
7                     False      True             False             False   
8                     False     False             False             False   
9                     False     False             False             False   
10                    False     False             False             False   
11                    False     False             False             False   

In [158]:
freq_groc = apriori(groc_df, min_support= 0.01, use_colnames = True)
freq_groc

,support,itemsets
0,0.033452,(UHT-milk)
1,0.017692,(baking powder)
2,0.052466,(beef)
3,0.033249,(berries)
4,0.026029,(beverages)
5,0.080529,(bottled beer)
6,0.110524,(bottled water)
7,0.064870,(brown bread)
8,0.055414,(butter)
9,0.027961,(butter milk)


In [160]:
association_rules(freq_groc, metric='lift').sort_values('lift',
                                                       ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
455,"(yogurt, whole milk)",(curd),0.056024,0.053279,0.010066,0.179673,3.372304,0.007081,1.154078
458,(curd),"(yogurt, whole milk)",0.053279,0.056024,0.010066,0.188931,3.372304,0.007081,1.163866
441,(root vegetables),"(citrus fruit, other vegetables)",0.108998,0.028876,0.010371,0.095149,3.295045,0.007224,1.073242
436,"(citrus fruit, other vegetables)",(root vegetables),0.028876,0.108998,0.010371,0.359155,3.295045,0.007224,1.390354
557,"(yogurt, other vegetables)",(whipped/sour cream),0.043416,0.071683,0.010168,0.234192,3.267062,0.007056,1.212206
560,(whipped/sour cream),"(yogurt, other vegetables)",0.071683,0.043416,0.010168,0.141844,3.267062,0.007056,1.114697
513,(root vegetables),"(other vegetables, tropical fruit)",0.108998,0.035892,0.012303,0.112873,3.144780,0.008391,1.086776
508,"(other vegetables, tropical fruit)",(root vegetables),0.035892,0.108998,0.012303,0.342776,3.144780,0.008391,1.355705
4,(root vegetables),(beef),0.108998,0.052466,0.017387,0.159515,3.040367,0.011668,1.127366
5,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628


In [ ]:
#### store_data.csv

In [163]:
store = list()
with open('data/store_data.csv','r') as fp:
    while True:
        line = fp.readline()
        if not line : break
        line = re.sub('\n','', line)
        store.append(line.split(','))
            

In [165]:
store_mat = te.fit_transform(store)
print(store_mat)


[[False  True  True ...  True False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False  True False]]


In [166]:
store_df = pd.DataFrame(store_mat, columns = te.columns_)
print(store_df)

       asparagus  almonds  antioxydant juice  asparagus  avocado  babies food  \
0          False     True               True      False     True        False   
1          False    False              False      False    False        False   
2          False    False              False      False    False        False   
3          False    False              False      False     True        False   
4          False    False              False      False    False        False   
5          False    False              False      False    False        False   
6          False    False              False      False    False        False   
7          False    False              False      False    False        False   
8          False    False              False      False    False        False   
9          False    False              False      False    False        False   
10         False    False              False      False    False        False   
11         False    False   

In [167]:
freq_store = apriori(store_df, min_support= 0.01, use_colnames = True)
freq_store

,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.010799,(barbecue sauce)
3,0.014265,(black tea)
4,0.011465,(body spray)
5,0.033729,(brownies)
6,0.087188,(burgers)
7,0.030129,(butter)
8,0.081056,(cake)
9,0.015331,(carrots)


In [168]:
association_rules(freq_store, metric='lift').sort_values('lift',
                                                       ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
232,(herb & pepper),(ground beef),0.049460,0.098254,0.015998,0.323450,3.291994,0.011138,1.332860
233,(ground beef),(herb & pepper),0.098254,0.049460,0.015998,0.162822,3.291994,0.011138,1.135410
404,"(mineral water, spaghetti)",(ground beef),0.059725,0.098254,0.017064,0.285714,2.907928,0.011196,1.262445
405,(ground beef),"(mineral water, spaghetti)",0.098254,0.059725,0.017064,0.173677,2.907928,0.011196,1.137902
416,"(mineral water, spaghetti)",(olive oil),0.059725,0.065858,0.010265,0.171875,2.609786,0.006332,1.128021
417,(olive oil),"(mineral water, spaghetti)",0.065858,0.059725,0.010265,0.155870,2.609786,0.006332,1.113898
209,(tomatoes),(frozen vegetables),0.068391,0.095321,0.016131,0.235867,2.474464,0.009612,1.183930
208,(frozen vegetables),(tomatoes),0.095321,0.068391,0.016131,0.169231,2.474464,0.009612,1.121381
205,(shrimp),(frozen vegetables),0.071457,0.095321,0.016664,0.233209,2.446574,0.009853,1.179825
204,(frozen vegetables),(shrimp),0.095321,0.071457,0.016664,0.174825,2.446574,0.009853,1.125268


In [169]:
### 유방암 data

In [170]:
breastCC = list()
with open('data/wisc_bc_data.csv','r') as fp:
    while True:
        line = fp.readline()
        if not line : break
        line = re.sub('\n','', line)
        breastCC.append(line.split(','))
        
breastCC_mat = te.fit_transform(breastCC)

breastCC_df = pd.DataFrame(breastCC_mat, columns = te.columns_)
breastCC_store = apriori(breastCC_df, min_support= 0.01, use_colnames = True)
association_rules(breastCC_store, metric='lift').sort_values('lift',
                                                       ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,"(""B"")",(12.36),0.626316,0.012281,0.012281,0.019608,1.596639,0.004589,1.007474
0,(0),"(""B"")",0.022807,0.626316,0.022807,1.000000,1.596639,0.008523,inf
1,"(""B"")",(0),0.626316,0.022807,0.022807,0.036415,1.596639,0.008523,1.014122
2,(0.1017),"(""B"")",0.010526,0.626316,0.010526,1.000000,1.596639,0.003934,inf
6,(0.1256),"(""B"")",0.010526,0.626316,0.010526,1.000000,1.596639,0.003934,inf
8,(0.1601),"(""B"")",0.010526,0.626316,0.010526,1.000000,1.596639,0.003934,inf
10,(12.36),"(""B"")",0.012281,0.626316,0.012281,1.000000,1.596639,0.004589,inf
12,(13.9),"(""B"")",0.010526,0.626316,0.010526,1.000000,1.596639,0.003934,inf
3,"(""B"")",(0.1017),0.626316,0.010526,0.010526,0.016807,1.596639,0.003934,1.006388
7,"(""B"")",(0.1256),0.626316,0.010526,0.010526,0.016807,1.596639,0.003934,1.006388
